# [Routing](markdown/routing-overview.md)

In [1]:
from dotenv import load_dotenv
import os

## [Part 10: Logical routing](markdown/logical-routing.md)

In [7]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# LLM with function call 
llm = ChatOllama(model="qwen2.5:32b-instruct-q4_K_M", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, choose one of the following datasources:
- 'python_docs' for Python-related questions
- 'js_docs' for JavaScript-related questions
- 'golang_docs' for Go-related questions.

Only respond with one of the above options."""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [8]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [9]:
print(result)

datasource='python_docs'


## Semantic routing

In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama import OllamaEmbeddings, ChatOllama

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OllamaEmbeddings(model='mxbai-embed-large')

llm = ChatOllama(model="llama3.1")

prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
Black holes! One of my favorite topics in physics.

A black hole is essentially a region in space where the gravitational pull is so strong that nothing, not even light, can escape once it gets too close. It's formed when a massive star collapses in on itself and its gravity becomes so intense that it warps the fabric of spacetime around it.

Imagine spacetime as a trampoline. If you put a heavy object on it, like a bowling ball, it will warp the surface, creating a curvature that affects nearby objects. A black hole is similar, but instead of just warping spacetime near its center, it creates such an intense gravitational field that anything that gets too close gets pulled inexorably towards the singularity at the center.

The point of no return around a black hole is called the event horizon. Once you cross the event horizon, you're trapped and can't escape, even if you were traveling at high speeds or had a powerful engine. It's as if the gravity of the black hole has 